## NLP With Supervised Learning
__August, 2018 - Christopher Sanchez__
__datasancheneer@gmail.com__

Hello, we have a lot to accomplish today so let's get started. Our goal today is to create a few models in order to determine whether or not we can predict which author wrote which book. We will be capitalizing on the books provided by nltk corupus known as gutenberg. Our goal is to achieve a test score above 90%. We will dive into some feature engineering in order to examine various information such as parts of speech, punctuation, and sentence length. We will then test our models on a new book to determine whether or not our model fits well when put to the test against a new set of text. Let's begin.

We will start by importing all of our necessary libraries. 

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import sys
import warnings
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn import ensemble
from sklearn.model_selection import train_test_split, cross_val_score

if not sys.warnoptions:
    warnings.simplefilter("ignore")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

All of our imports are successfully imported. Now we will import our books and clean the text using regular expression.

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

Perfect. Let's parse the data using spaCy, group the data into sentences, and create our dataframe.

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


We have successfully cleaned the text, grouped our sentences, created a dataframe and our data is looking good. Let's create our bag of words function. We will add the various columns necessary for feature engineering. Finally let's create a set of unique words.

In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0    
    df['exclamation_count'] = 0
    df['verb_count'] = 0
    df['adverb_count'] = 0
    df['noun_count'] = 0
    df['pronoun_count'] = 0
    df['adjective_count'] = 0

    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)
# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)


Now that we have that out of the way. Let's move on to creating a dataframe that includes the common, unique, words.

In [6]:
word_counts = bow_features(sentences, common_words)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


Perfect our bag of words function performed nicely. Time to start some feature engineering. We will start with determining the length of each sentence, the sum of common words for each row, and the amount of exclamation marks.

In [7]:
column_list = word_counts.drop(['text_sentence','text_source'], 1)
columns = column_list.columns
word_counts['sentence_len'] = word_counts['text_sentence'].str.len()
word_counts['common_sum'] = word_counts[columns].sum(axis=1)

for i in range(len(word_counts['text_sentence'])):
    count = word_counts['text_sentence'][i].text.count('!') 
    word_counts['exclamation_count'][i] = count


Nicely done. Parts of speech is important. We will get a count of the various parts of speech, starting with verbs below.

In [8]:
for i in range(len(word_counts['text_sentence'])):
    sentence = word_counts['text_sentence'][i]
    count_vbz = 0
    
    for token in word_counts['text_sentence'][i]:
        count_vbz = count_vbz + token.tag_.count('VBZ') + token.tag_.count('VBG') + token.tag_.count('VBD') + token.tag_.count('VB') + token.tag_.count('BES') + token.tag_.count('HVS') + token.tag_.count('MD') + token.tag_.count('HVS') + token.tag_.count('VBN') + token.tag_.count('VBP')
    word_counts['verb_count'][i] = count_vbz 

Adverbs:

In [9]:
for i in range(len(word_counts['text_sentence'])):
    sentence = word_counts['text_sentence'][i]
    count_adverbs = 0
    
    for token in word_counts['text_sentence'][i]:
        count_adverbs = count_adverbs + token.tag_.count('EX') + token.tag_.count('RB') + token.tag_.count('RBR') + token.tag_.count('RBS') + token.tag_.count('Rp') + token.tag_.count('WRB')
    word_counts['adverb_count'][i] = count_adverbs 

Nouns:

In [10]:
for i in range(len(word_counts['text_sentence'])):
    sentence = word_counts['text_sentence'][i]
    count_nouns = 0
    
    for token in word_counts['text_sentence'][i]:
        count_nouns = count_nouns + token.tag_.count('NN') + token.tag_.count('NNS') + token.tag_.count('WP')
    word_counts['noun_count'][i] = count_nouns 

Pronouns:

In [11]:
for i in range(len(word_counts['text_sentence'])):
    sentence = word_counts['text_sentence'][i]
    count_pronouns = 0
    
    for token in word_counts['text_sentence'][i]:
        count_pronouns = count_pronouns + token.tag_.count('PRP') + token.tag_.count('PRP$')
    word_counts['pronoun_count'][i] = count_pronouns 

Adjectives:

In [12]:
for i in range(len(word_counts['text_sentence'])):
    sentence = word_counts['text_sentence'][i]
    count_adjective = 0
    
    for token in word_counts['text_sentence'][i]:
        count_adjective = count_adjective + token.tag_.count('JJ') + token.tag_.count('JJR') + token.tag_.count('JJS')
    word_counts['adjective_count'][i] = count_adjective 

Our feature engineering is complete. I wonder what our data looks like now.

In [13]:
word_counts.head()

,follow,croquet,recommend,sickness,high,duchess,leader,gentleman,smith,accommodate,custody,bottle,command,vigour,benwick,tax,tiptoe,undoubtedly,hedgehog,impossible,recognise,partiality,length,surprised,active,send,fourth,player,i.,soup,doubtfully,enquire,estate,song,penny,wentworth,collar,modest,impatient,evident,landlord,lizard,will,affront,russell,thousand,suppose,entertain,drown,paint,quite,crouch,inclined,meal,quickly,frederick,aloud,longer,suspend,party,school,minutia,sorrow,pass,appearance,invent,climb,richard,shade,encouraging,spar,fifteenth,officer,announce,unkind,pleased,asleep,nasty,willing,alacrity,pebble,compare,bench,altogether,range,tomorrow,hamilton,affectionately,farmer,mamma,sharply,declare,suspense,retrenchment,pleasanter,slowly,wash,show,perception,late,idiotic,lie,glance,simply,engine,dine,torment,bear,fry,resist,oppose,run,ink,cordial,trouble,puss,accomplishment,yes,knee,treading,naval,snug,snappishly,oyster,finger,alteration,porpoise,happiness,turtle,heap,look,odd,off,freckle,fun,civil,hastily,tread,mystery,servant,arrum,emphasis,pretext,piteous,indulge,resident,ring,nibble,usually,sluggard,hurry,tremble,release,young,intimately,doubtful,fare,correct,interesting,ugh,would,require,rapid,frightful,gifted,grave,interrupt,have,two,worry,rub,spectacle,cultivate,agree,interval,unlike,understanding,sense,caterpillar,he,jemima,flatter,portion,dress,intelligence,consequently,afford,tie,talent,dispute,little,tired,bite,acre,help,morning,live,contrive,irritated,fender,hart,curly,nay,liberty,nothingness,somersetshire,dish,urge,seal,consciousness,boon,quick,cheerfully,hospitality,misfortune,invalid,drawl,equally,meaning,inward,oop,presently,signify,tremulous,recollect,loving,calculation,providence,liking,grace,uncommonly,opening,five,reconciliation,mortify,day,profess,axis,maybe,fur,sensation,anger,chuckle,taunton,fool,forty,market,consider,club,canvas,...,reason,education,spot,dressed,squeak,alice,idiot,curtsey,companion,happily,amusement,fan,beau,provoke,cobb,intelligible,month,farth,long,elizabeth,eager,me,draggled,keener,lead,illness,hot,half,attachment,sport,discovery,soo,waistcoat,size,bring,move,succeed,latitude,drawing,before,subdued,uneasy,temper,hurt,mercia,principally,esteem,clay,esq,twinkle,imprudence,afore,despair,short,forget,stretch,air,unworthy,conjecture,solitude,shoes,treat,narrow,should,appear,comfortable,pun,yawn,fairy,behead,toast,unite,majesty,interest,unjust,camomile,breath,lesson,their,persuasion,discredit,oneself,they,robinson,new,blade,patiently,guinea,mrs,benefit,delightful,twelfth,clothe,wild,fearful,curl,nor,shower,repose,spend,steam,betray,lady,confusing,rude,juror,heavy,attempt,always,permit,vote,promise,listen,grumble,complete,accordingly,polite,ease,stupid,recover,height,classic,decision,another,grammar,spirit,spending,particularly,house,issue,everything,shark,baronetage,merely,onion,muchness,either,joint,magpie,owner,completely,distress,flash,apprehension,endeavour,division,picture,gift,upon,shore,`,because,dozen,'s,wallis,there,study,nearer,tillie,fork,consultation,allude,by,nice,mine,old,proper,altered,exquisite,owe,execute,civility,disagreeable,est,housemaid,remain,misery,plymouth,easy,penance,worldly,hold,lazy,exertion,dare,shepherd,rap,pull,gentle,hjckrrh,dog,chance,protection,bright,deep,conference,effort,chrysalis,grow,pack,henrietta,standard,gardener,breeding,laughing,charitable,time,argue,solid,affliction,instance,absolutely,awkward,brave,apparent,peculiarly,amidst,why,languid,mouth,lonely,which,decent,waiting,affectionate,goldfish,county,possession,value,regret,building,washing,shed,slip,wretched,pleasant,pick,impatiently,moderate,uglification,text_sentence,text_source,exclamation_count,verb_count,adverb_count,noun_count,pronoun_count,adjective_count,sentence_len,common_sum
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

The data is looking good. We can start modeling the data now. We will use various classifiers to try to determine which one works the best. Random forest is one of my favorites and that's what we will start with.

In [14]:
rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))
print('\nCross Val score:',cross_val_score(rfc, X_test, y_test, cv=5))


Training set score: 0.9918495297805643

Test set score: 0.8834586466165414

Cross Val score: [0.84309133 0.8685446  0.85411765 0.86117647 0.85411765]


Our test score was only 88% not quite the 90 we are looking for. Will our next model, logistic regression, get us there? Let's take a look.

In [15]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))
print('\nCross Val score:',cross_val_score(lr, X_test, y_test, cv=5))


(3190, 3070) (3190,)
Training set score: 0.964576802507837

Test set score: 0.9201127819548872

Cross Val score: [0.93911007 0.89671362 0.88941176 0.88705882 0.92235294]


Awesome! 92% It did however bounce a bit in our cross val score. Boosted trees is nice. Let's see what it can do.

In [16]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))
print('\nCross Val score:',cross_val_score(clf, X_test, y_test, cv=5))


Training set score: 0.8931034482758621

Test set score: 0.875

Cross Val score: [0.87587822 0.85915493 0.86588235 0.86352941 0.86823529]


Disappointing! Our lowest score yet! However it did remain consistent which is nice. SVC has to do decent right?

In [17]:
svc = SVC()
train = svc.fit(X_train, y_train)

print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))
print('\nCross Val score:',cross_val_score(svc, X_test, y_test, cv=5))


Training set score: 0.6824451410658308

Test set score: 0.6917293233082706

Cross Val score: [0.69086651 0.69248826 0.69176471 0.69176471 0.69176471]


Ouch! only 69% svc did horribly. Time to move on. Neural networks are usually very powerful, though computationally extensive. We'll start with the relu activation function.

In [18]:
mlp_relu = MLPClassifier(hidden_layer_sizes=(1000, 1000, 1000, 1000, 1000), activation='relu')
train = mlp_relu.fit(X_train, y_train)

print('Training set score:', mlp_relu.score(X_train, y_train))
print('\nTest set score:', mlp_relu.score(X_test, y_test))
print('\nCross Val score:',cross_val_score(mlp_relu, X_test, y_test, cv=5))


Training set score: 0.9971786833855799

Test set score: 0.9224624060150376

Cross Val score: [0.92974239 0.91549296 0.92941176 0.88       0.93647059]


This is a nice model. It has provided our best score, and fit pretty consistently across our cross validation score. Tanh is very similar to relu, so we should be expecting a similar result next.

In [19]:
mlp_tanh = MLPClassifier(hidden_layer_sizes=(1000, 1000, 1000, 1000, 1000), activation='tanh')
train = mlp_tanh.fit(X_train, y_train)

print('Training set score:', mlp_tanh.score(X_train, y_train))
print('\nTest set score:', mlp_tanh.score(X_test, y_test))
print('\nCross Val score:',cross_val_score(mlp_tanh, X_test, y_test, cv=5))


Training set score: 0.990909090909091

Test set score: 0.9140037593984962

Cross Val score: [0.92505855 0.9084507  0.89882353 0.88       0.91529412]


This fit nicely as well, however not as well as our relu neural network model. Let's check our the logistic activation function next.

In [20]:
mlp_logistic = MLPClassifier(hidden_layer_sizes=(1000, 1000, 1000, 1000, 1000), activation='logistic')
train = mlp_logistic.fit(X_train, y_train)

print('Training set score:', mlp_logistic.score(X_train, y_train))
print('\nTest set score:', mlp_logistic.score(X_test, y_test))
print('\nCross Val score:',cross_val_score(mlp_logistic, X_test, y_test, cv=5))


Training set score: 0.987460815047022

Test set score: 0.9163533834586466

Cross Val score: [0.69086651 0.69248826 0.69176471 0.69176471 0.69176471]


It's interesting to see that we received a 91% on our test set, but only an average of 69% across our cross validation score. This is a horrible model. 

Our two best models were our logistic regression model, and our neural network model utilizing the relu activation function. We will now import a new book in order to test against Alice In Wonderland to determine whether or not our two best models will work with next texts. We will use Buster Brown by Burgess.

We will repeat the process as above, and clean the text, and create our bag of words features.

In [21]:
busterbrown = gutenberg.raw('burgess-busterbrown.txt')
busterbrown = re.sub(r'\b[A-Z]+\b', '', busterbrown)
busterbrown = text_cleaner(busterbrown)
busterbrown_doc = nlp(busterbrown)
# Group into sentences.
busterbrown_sents = [[sent, "Burgess"] for sent in busterbrown_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + busterbrown_sents)

busterbrownwords = bag_of_words(busterbrown_doc)

common_words_test = set(alicewords + busterbrownwords)

burgess_test = bow_features(sentences, common_words_test)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500


In [22]:
column_list = burgess_test.drop(['text_sentence','text_source'], 1)
columns = column_list.columns
burgess_test['sentence_len'] = burgess_test['text_sentence'].str.len()
burgess_test['common_sum'] = burgess_test[columns].sum(axis=1)

for i in range(len(burgess_test['text_sentence'])):
    count = burgess_test['text_sentence'][i].text.count('!') 
    burgess_test['exclamation_count'][i] = count

    
for i in range(len(burgess_test['text_sentence'])):
    sentence = burgess_test['text_sentence'][i]
    count_vbz = 0
    
    for token in burgess_test['text_sentence'][i]:
        count_vbz = count_vbz + token.tag_.count('VBZ') + token.tag_.count('VBG') + token.tag_.count('VBD') + token.tag_.count('VB') + token.tag_.count('BES') + token.tag_.count('HVS') + token.tag_.count('MD') + token.tag_.count('HVS') + token.tag_.count('VBN') + token.tag_.count('VBP')
    burgess_test['verb_count'][i] = count_vbz 
    
    
for i in range(len(burgess_test['text_sentence'])):
    sentence = burgess_test['text_sentence'][i]
    count_adverbs = 0
    
    for token in burgess_test['text_sentence'][i]:
        count_adverbs = count_adverbs + token.tag_.count('EX') + token.tag_.count('RB') + token.tag_.count('RBR') + token.tag_.count('RBS') + token.tag_.count('Rp') + token.tag_.count('WRB')
    burgess_test['adverb_count'][i] = count_adverbs 
    
    
for i in range(len(burgess_test['text_sentence'])):
    sentence = burgess_test['text_sentence'][i]
    count_nouns = 0
    
    for token in burgess_test['text_sentence'][i]:
        count_nouns = count_nouns + token.tag_.count('NN') + token.tag_.count('NNS') + token.tag_.count('WP')
    burgess_test['noun_count'][i] = count_nouns 
    

for i in range(len(burgess_test['text_sentence'])):
    sentence = burgess_test['text_sentence'][i]
    count_pronouns = 0
    
    for token in burgess_test['text_sentence'][i]:
        count_pronouns = count_pronouns + token.tag_.count('PRP') + token.tag_.count('PRP$')
    burgess_test['pronoun_count'][i] = count_pronouns 
    

for i in range(len(burgess_test['text_sentence'])):
    sentence = burgess_test['text_sentence'][i]
    count_adjective = 0
    
    for token in burgess_test['text_sentence'][i]:
        count_adjective = count_adjective + token.tag_.count('JJ') + token.tag_.count('JJR') + token.tag_.count('JJS')
    burgess_test['adjective_count'][i] = count_adjective 

Perfect that's done. Now we will test our models starting with our neural network utilizing the relu activation function.

In [23]:
mlp_relu = MLPClassifier(hidden_layer_sizes=(1000, 1000, 1000, 1000, 1000), activation='relu')
Y = burgess_test['text_source']
X = np.array(burgess_test.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = mlp_relu.fit(X_train, y_train)

print('Training set score:', mlp_relu.score(X_train, y_train))
print('\nTest set score:', mlp_relu.score(X_test, y_test))
print('\nCross Val score:',cross_val_score(mlp_relu, X_test, y_test, cv=5))

Training set score: 0.9962287869264613

Test set score: 0.9161168708765316

Cross Val score: [0.84507042 0.85915493 0.88732394 0.86729858 0.87203791]


We achieved a test score of nearly 92%, but it didn't perform as well in our cross validation score. What about the logistic regression model?

In [24]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))
print('\nCross Val score:',cross_val_score(lr, X_test, y_test, cv=5))


(1591, 2341) (1591,)
Training set score: 0.9767441860465116

Test set score: 0.9048067860508954

Cross Val score: [0.88732394 0.89671362 0.87323944 0.87203791 0.83886256]


Our logistic regression function achieved a 90% on our test score, like our neural network, it didn't perform so hot in the cross validation score.

### Discussion and Conclusion

Our models did pretty well. We achieved the 90% in the test scores we were looking to achieve. There may be some more feature engineering we can do to better fit our data, like checking other punctuation, and looking at the context of the sentences. I learned a lot, and I hope you did to. 